In [35]:
import numpy as np
import pandas as pd

In [36]:
data = pd.read_csv('train.csv', header = 0)

In [37]:
display(data.head(10))

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [38]:
data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [39]:
n_records = data.shape[0]
labels_survived = data[data['Survived'] == 1].shape[0]
labels_not_survived = data[data['Survived'] == 0].shape[0]
labels_other = data[(data['Survived'] != 1) & (data['Survived'] != 0)].shape[0]


print("Total number of records: %d" % n_records)
print("Passengers survived: %d" % labels_survived)
print("Passengers didn't survive: %d" % labels_not_survived)
print("Passengers uncertain: %d" % labels_other)


Total number of records: 891
Passengers survived: 342
Passengers didn't survive: 549
Passengers uncertain: 0


In [43]:
labels = data['Survived']
features = data.drop('Survived', axis = 1)

In [44]:
data_prep = features.drop(['Name', 'Ticket', 'Cabin', 'Embarked'], axis = 1)
display(data_prep.head())

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare
0,1,3,male,22.0,1,0,7.2500
1,2,1,female,38.0,1,0,71.2833
2,3,3,female,26.0,0,0,7.9250
3,4,1,female,35.0,1,0,53.1000
4,5,3,male,35.0,0,0,8.0500


In [45]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler() 
numerical = ['Age', 'Fare', 'Pclass', 'SibSp', 'Parch']

data_norm = pd.DataFrame(data = data_prep)
data_norm[numerical] = scaler.fit_transform(data_norm[numerical])

display(data_norm.head())

/usr/local/lib/python3.7/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare
0,1,1.0,male,0.271174,0.125,0.0,0.014151
1,2,0.0,female,0.472229,0.125,0.0,0.139136
2,3,1.0,female,0.321438,0.000,0.0,0.015469
3,4,0.0,female,0.434531,0.125,0.0,0.103644
4,5,1.0,male,0.434531,0.000,0.0,0.015713


In [48]:
data_final = pd.get_dummies(data_norm)
encoded = list(data_final.columns)
print("%d total features after one-hot encoding." % len(encoded))

8 total features after one-hot encoding.


In [49]:
display(data_final.head())

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male
0,1,1.0,0.271174,0.125,0.0,0.014151,0,1
1,2,0.0,0.472229,0.125,0.0,0.139136,1,0
2,3,1.0,0.321438,0.000,0.0,0.015469,1,0
3,4,0.0,0.434531,0.125,0.0,0.103644,1,0
4,5,1.0,0.434531,0.000,0.0,0.015713,0,1
